In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d salader/dogs-vs-cats

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout

In [ ]:
train_ds = keras.utils.image.image_dataset_from_directory(
    directory = '/content/train', #path to train data folder where images for training are there
    labels='inferred',
    label_mode = 'int', #labels cat images as 0 and dog images as 1. so, when model predicts, it gives answer in 0 and 1.
    batch_size=32,
    image_size=(256,256)
)

#same for validation data images
validation_ds = keras.utils.image.image_dataset_from_directory(
    directory = '/content/test',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(256,256)
)

In [ ]:
'''
the function below converts the images into the array of floating numbers, and also scales the values between 0 and 1.
'''
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

#the above function is used for converting the images of train and test data into array of floating values between 0 and 1.
train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [ ]:
model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds, epochs=10, validation_data=validation_ds)

In [ ]:
import cv2

In [ ]:
test_img = cv2.imread('/content/dog.jpg') #taking one dog image as the test image
plt.imshow(test_img) #for just displaying the dog image

In [ ]:
test_img = cv2.resize(test_img,(256,256))
#NOTE:- we always pass our images in form of batches. as we are just dealing with one test image right now, we wrote 1. also, as the image is rgb, we wrote 3.
#all in all, we created a "tensor of 256x256 rgb image with batch size 1"
test_input = test_img.reshape((1,256,256,3))

In [ ]:
model.predict(test_input)
#output = 1

In [ ]:
#----------DATA AUGMENTATION-----------

In [ ]:
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
img = image.load_img('train/cat_18.jpg',target = (200,200))
#we are just taking one image right now

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img)

In [ ]:
type(img)
#PIL.Image.Image

In [ ]:
#below object called datagen is used for image augmentation
datagen = ImageDataGenerator(
    rotation_range=40, #rotates to 40 degrees
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2, #distorting the image
    zoom_range=0.2,
    horizontal_flip=True, #cat on left comes to right when we flip img horizonatally
)

In [ ]:
img = image.img_to_array(img) #converting into array of values

In [ ]:
img.shape
#(200,200,3)

In [ ]:
input_batch = img.reshape(1,200,200,3) #created a tensor

In [ ]:
i=0
#when we are working with just one image, we use datagen.flow and if we work with multiple images stored in folder, we use datagen.flow_from_dir
#save_to_dir creates a new folder called aug where all these 10 augmented images from a single image, gets stored.
for output in datagen.flow(input_batch,batch_size=1,save_to_dir='aug'):
  i+=1
  if i == 10:
    break

In [ ]:
input_batch.shape

In [ ]:
#WHEN DEALING WITH WHOLE DIRECTORY OF IMAGES:-

In [ ]:
batch_size = 16

#we created seperate variables for train and validation data and initiated the type of transformation we want to apply.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
#for test data, only rescaling required. no additional transformations required.
test_datagen = ImageDataGenerator(
    rescale=1./255
)

#applying transformation to the train and test data, with 16 images at a time.
train_generator = train_datagen.flow_from_directory(
    '/content/train',
    target_size=(150,150),
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    '/content/test',
    target_size=(150,150),
    batch_size=batch_size,
    class_mode='binary'
)


#the CNN MODEL comes here below where you add all the convolution layers and fully connected layers
#compile model
#train model:-
model.fit(train_generator, epochs=25, validation_data=validation_generator)
'''
we only use the transformed images for training and leave the original images aside.
'''